In [669]:
import math, scipy.special
x = math.sqrt(13)
phi = (2+x)/3
cphi = (2-x)/3
phi = (2+x)/3
cphi = (2-x)/3
c0 = (7+2*x)/(4*x)
c1 = (-7+2*x)/(4*x)
psi = (1-3*phi)
cpsi = (1-3*cphi)
gamma = 31/9

In [670]:
def divrem(b,d):
    if d != 0:
        e = b//d
        f = b % d
    else:
        e = 0
        f = 0
    if e >= 3:
        e1 = 3
        f1 = 0
    else:
        e1 = e
        f1 = f
    return e, f, e1, f1
def depthadjust(d, e, f):
    b = d*e+f
    e0,f0,e1,f1 = divrem(b, d)
    return e1,f1
def addone(d, e, f):
    b = e*d+f+1
    e0,f0,e1,f1 = divrem(b, d)
    return e1, f1

In [671]:
def alpha(e):
    return (3-e)/2+2*e/3
def computep3e3(d, a, e, f):
    return 2**d
def computep3a0(d, a, e, f):
    return ((3/2)**d)*(((9+e)/9)**d)*(((10+e)/(9+e))**f)
def computep3d0(d, a, e, f):
    return 1
def computep3d1(d, a, e, f):
    return alpha(e)
def computep3d2(d, a, e, f):
    s = 0
    if a == 0:
        s =  computep3a0(d, a, e, f)
    elif e == 3:
        s =  computep3e3(d, a, e, f)
    elif e == 2:
        if f == 1:
            s =  2*alpha(e)
        elif f == 0:
            s =  2*1/2 + 4*alpha(e)/3
    elif e == 1:
        a1 = min(a, 2)
        s = 2*alpha(e+f)/3 + a1*alpha(e+f+1)/2 + (3-e-a1)*alpha(e+f)/2
    elif e == 0:
        a1 = min(3, a)
        s = (3-a1)*alpha(e+f)/2 + a1*alpha(e+f+1)/2
    else:
        print(e, "bad value of e")
    return s

In [672]:
def adjust(d, e, f, delta):
    b = d*e + f + delta
    e1, f1, e2, f2 = divrem(b, d)
    return e2, f2

In [673]:
def computep3d3(d, a, e, f):
    s = 0
    e1, f1 = depthadjust(d-1, e, f)
    e2, f2 = addone(d-1, e, f)
    #print(e1, f1, e2, f2)
    if a == 0:
        s =  computep3a0(d, a, e, f)
    elif e == 3:
        s =  computep3e3(d, a, e, f)
    elif e == 2:
        if  f == 0:
            s = 58.0/9
        else:
            s = s + 4*computep3d2(d-1, a-1, e1, f1)/3
            s = s + computep3d2(d-1, a-1, e2, f2)/2
    elif e == 1:
        a1 = min(a, 2)
        safe = max(0, 3-a1-e)
        s = s + a1*computep3d2(d-1, a-a1, e2, f2)/2
        s = s + (safe/2+2*e/3)*computep3d2(d-1, a-a1, e1, f1)
    elif e == 0:
        a1 = min(a, 3)
        safe = max(0, 3-a1-e)
        s = s + a1*computep3d2(d-1, a-a1, e2, f2)/2
        s = s + (safe/2+2*e/3)*computep3d2(d-1, a-a1, e1, f1)
    else:
        print(e, "bad value of e in computep3d3")
    return s

In [674]:
def ssum(factor, lower, upper):
    sum = 0
    i = 1
    t = 1
    while (i <= upper):
        t = t*factor
        sum = sum + t*scipy.special.binom(lower+upper-i, lower)
        i += 1
    return sum
def middlep3(l,r):
    r43 = (4/3)**r
    blr = scipy.special.binom(l+r,l)
    s0 = r43*blr
    s1 = c0*(1-4/(3*phi))*ssum(3*phi/4, l, r)
    s2 = c1*(1-4/(3*cphi))*ssum(3*cphi/4, l, r)
    s3 = (1-1/(gamma))*ssum(gamma, r, l)
    s = s0 + s1 + s2 + s3
    return s

In [675]:
def computep3(d, a, e, f):
    if d == 0:
        s = computep3d0(d, a, e, f)
    elif d == 1:
        s = computep3d1(d, a, e, f)
    elif d == 2:
        s = computep3d2(d, a, e, f)
    elif d == 3:
        s = computep3d3(d, a, e, f)
    elif a == 0:
        s = computep3a0(d, a, e, f)
    elif a == 1:
        corr = e
        unsafe = min(3-e, 1)
        safe = 3- unsafe - corr
        e1, f1 = depthadjust(d-1, e, f)
        e2, f2 = addone(d-1, e1, f1)
        s = (unsafe/2)*computep3(d-1, a-1, e2, f2)
        s = s + (safe/2+ 2*corr/3)*computep3(d-1, a-1, e1, f1)
    elif e == 3:
        s = computep3e3(d, a, e, f)
    elif e == 2:
        if f >= 1:
            s = (2**f)*computep3(d-f, a, e, 0)           
        elif d <= a-1:
            s = (c0*(phi**d) + c1*(cphi**d))
        elif d>= (4-e)*a:
            two = ((3-e)*alpha(e+1)/2 + 2*e*alpha(e)/3)
            s = (two**a)*alpha(e)**(d-2*a)
        else:
            l = d-a
            r = 2*a-d
            s = middlep3(l,r)
    return s

In [676]:
def test(d,a,b):
    i = 0
    while (i<=d):
        j = 0
        while (j<=b):
            k = 0
            e0, f0, e, f = divrem(j, i)
            while (k <= a):
                s = computep3(i,k,e,f)
                print(i,k,j,e,f,s)
                k += 1  
            j += 1
        i += 1     

In [677]:
def gen(d, a, e, f):
    options = list()
    b = d*e + f
    for us in range(4):
        unsafe = min(a, us)
        for cr in range(3-unsafe+1):
            corr = min(cr, b)
            safe = max(0, 3-unsafe-corr)
            delta = e - corr
            e1, f1 = adjust(d-1, e, f, delta)
            e2, f2 = addone(d-1, e1, f1)
            item = (unsafe, safe, corr, e1, f1, e2, f2)
            if item not in options:
                options.append(item)
    return options

In [678]:
a0 = 5
d = 4
b = 6
#test(d,a,b)
for e in range(4):
    for f in range(d-1):
        b = d*e + f
        if e >= 2:
            for a in range(a0):
                s = computep3(d, a, e, f)
                options  = gen(d, a, e, f)
                for item in options:
                    (unsafe, safe, corr, e1, f1, e2, f2) = item
                    t = 0
                    t1 = safe*computep3(d-1, a-unsafe, e1, f1)/2
                    t2 = corr*2*computep3(d-1, a-unsafe, e1, f1)/3
                    t3 = unsafe*computep3(d-1, a-unsafe, e2, f2)/2
                    t = t1  + t2 + t3
                    if s - t < -10**(-13):
                        print("d=",d, "a=",a, "e=",e, "f=",f, "u=",unsafe, "s=",safe, "c=",corr, "a'=",a-unsafe, "e1=",e1, "f1=",f1, "e2=",e2, "f2=",f2, "s=",s, "t=",t, t1, t2, t3)

d= 4 a= 1 e= 2 f= 0 u= 0 s= 1 c= 2 a'= 1 e1= 2 f1= 0 e2= 2 f2= 1 s= 11.577160493827162 t= 11.814814814814817 3.2222222222222223 8.592592592592593 0.0
d= 4 a= 2 e= 2 f= 0 u= 1 s= 0 c= 2 a'= 1 e1= 2 f1= 0 e2= 2 f2= 1 s= 11.864197530864196 t= 12.037037037037038 0.0 8.592592592592593 3.4444444444444446
d= 4 a= 3 e= 2 f= 0 u= 0 s= 3 c= 0 a'= 3 e1= 2 f1= 2 e2= 3 f2= 0 s= 9.090277777777779 t= 11.0 11.0 0.0 0.0
d= 4 a= 3 e= 2 f= 0 u= 0 s= 2 c= 1 a'= 3 e1= 2 f1= 1 e2= 2 f2= 2 s= 9.090277777777779 t= 11.481481481481481 6.888888888888888 4.592592592592593 0.0
d= 4 a= 3 e= 2 f= 0 u= 0 s= 1 c= 2 a'= 3 e1= 2 f1= 0 e2= 2 f2= 1 s= 9.090277777777779 t= 11.814814814814817 3.2222222222222223 8.592592592592593 0.0
d= 4 a= 3 e= 2 f= 0 u= 0 s= 0 c= 3 a'= 3 e1= 1 f1= 2 e2= 2 f2= 0 s= 9.090277777777779 t= 11.925925925925924 0.0 11.925925925925924 0.0
d= 4 a= 3 e= 2 f= 0 u= 1 s= 2 c= 0 a'= 2 e1= 2 f1= 2 e2= 3 f2= 0 s= 9.090277777777779 t= 11.333333333333332 7.333333333333333 0.0 4.0
d= 4 a= 3 e= 2 f= 0 u= 1 s=

(248/54)*(4/3)

In [680]:
556/54

10.296296296296296